In [11]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf

from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer


In [12]:
## 
#1. 당뇨병 데이터에서 x_train y_train 캡처
#2. lightgbm 모델링 평가까지 평가결과출력캡처
#3. LogisRegression 모델링 평가까지 평과결과출력캡처
#
#accuracy_score, f1_score, precision_score, recall_score
#4개 나열하고 평가 결과 출력



## 당뇨병 데이터에서 x_train, y_train 캡처


In [13]:
file_path = 'D:\Code\당뇨병/diabetes_binary_5050split_health_indicators_BRFSS2015.csv'

df = pd.read_csv(file_path)

print(df)


       Diabetes_binary  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0                  0.0     1.0       0.0        1.0  26.0     0.0     0.0   
1                  0.0     1.0       1.0        1.0  26.0     1.0     1.0   
2                  0.0     0.0       0.0        1.0  26.0     0.0     0.0   
3                  0.0     1.0       1.0        1.0  28.0     1.0     0.0   
4                  0.0     0.0       0.0        1.0  29.0     1.0     0.0   
...                ...     ...       ...        ...   ...     ...     ...   
70687              1.0     0.0       1.0        1.0  37.0     0.0     0.0   
70688              1.0     0.0       1.0        1.0  29.0     1.0     0.0   
70689              1.0     1.0       1.0        1.0  25.0     0.0     0.0   
70690              1.0     1.0       1.0        1.0  18.0     0.0     0.0   
70691              1.0     1.0       1.0        1.0  25.0     0.0     0.0   

       HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0 

In [14]:
X_data = df.drop(columns='Diabetes_binary')
y_data = df.loc[:,'Diabetes_binary']

print(f'X_data_confirmed : \n{X_data}')

print('*'*80)

print(f'y_data_confirmed: \n{y_data}')

print('*'*80)

X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    test_size = 0.8,
                                                    random_state=0,
                                                    stratify=y_data)

print(f'X_train의 값 : \n{X_train}')

print('*'*80)

print(f'y_train의 값 : \n{y_train}')

X_data_confirmed : 
       HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0         1.0       0.0        1.0  26.0     0.0     0.0   
1         1.0       1.0        1.0  26.0     1.0     1.0   
2         0.0       0.0        1.0  26.0     0.0     0.0   
3         1.0       1.0        1.0  28.0     1.0     0.0   
4         0.0       0.0        1.0  29.0     1.0     0.0   
...       ...       ...        ...   ...     ...     ...   
70687     0.0       1.0        1.0  37.0     0.0     0.0   
70688     0.0       1.0        1.0  29.0     1.0     0.0   
70689     1.0       1.0        1.0  25.0     0.0     0.0   
70690     1.0       1.0        1.0  18.0     0.0     0.0   
70691     1.0       1.0        1.0  25.0     0.0     0.0   

       HeartDiseaseorAttack  PhysActivity  Fruits  Veggies  ...  \
0                       0.0           1.0     0.0      1.0  ...   
1                       0.0           0.0     1.0      0.0  ...   
2                       0.0           1.0     1.0      1.0

### lightgbm 모델링 평가까지 평가결과출력캡처


In [15]:
from sklearn.model_selection import GridSearchCV #교차검증
from lightgbm import LGBMClassifier

In [16]:
# XBoost와 LightGBM은 비슷하다!
base_lgbm = LGBMClassifier(random_state=0)

params = {'n_estimators' : [50,100,200],
          'learning_rate' : [0.05, 0.1, 0.5],
          'max_depth':[3,4,5]
}

grid_lgbm = GridSearchCV(
    estimator = base_lgbm,
    param_grid=params,
    scoring='accuracy',
    cv=10
)

grid_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 6362, number of negative: 6362
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 180
[LightGBM] [Info] Number of data points in the train set: 12724, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 6362, number of negative: 6362
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000800 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 180
[LightGBM] [Info] Number of data points in the train set: 12724, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


GridSearchCV(cv=10, estimator=LGBMClassifier(random_state=0),
             param_grid={'learning_rate': [0.05, 0.1, 0.5],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [17]:
#최적의 성능 파라미터 생성

print(grid_lgbm.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [18]:
# best 모델 생성
best_lgbm = LGBMClassifier(learning_rate=0.1, 
                           max_depth=4,
                           n_estimators=200,
                           random_state=0)

In [19]:
best_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 7069, number of negative: 7069
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 181
[LightGBM] [Info] Number of data points in the train set: 14138, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

LGBMClassifier(max_depth=4, n_estimators=200, random_state=0)

In [20]:
pred_test = best_lgbm.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def eval_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    data = [[accuracy, f1, precision, recall]]
    columns = ['accuracy', 'f1_score', 'precision', 'recall']
    eval_df = pd.DataFrame(data=data, columns=columns)
    return eval_df

result = eval_model(y_test, pred_test)

print(f'평가용 데이터에 대한 성능 평가 결과 : \n{result}')

평가용 데이터에 대한 성능 평가 결과 : 
   accuracy  f1_score  precision    recall
0  0.750504  0.761422   0.729499  0.796266


In [22]:
accuracy_test = accuracy_score(y_test, pred_test)

print(f'평가용 데이터에 대한 정확도 : {accuracy_test}')

평가용 데이터에 대한 정확도 : 0.7505039431339958


### LogisticRegression 모델링 평가까지 평가결과출력캡처


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

base_lgr = LogisticRegression(random_state=0)

number1 = np.arange(0.1, 3.1)
params = {'C':number1, 'max_iter': [5000, 10000]}

grid_lgr = GridSearchCV(
    estimator=base_lgr,
    param_grid=params,
    scoring='accuracy',
    cv=10
)

grid_lgr.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=0),
             param_grid={'C': array([0.1, 1.1, 2.1]),
                         'max_iter': [5000, 10000]},
             scoring='accuracy')

In [24]:
print(grid_lgr.best_params_)

{'C': 0.1, 'max_iter': 5000}


In [25]:
best_lgr = LogisticRegression(C=1.3, max_iter=5000, random_state=0)

In [26]:
best_lgr.fit(X_train, y_train)

LogisticRegression(C=1.3, max_iter=5000, random_state=0)

In [27]:
pred_test = best_lgr.predict(X_test)
print(pred_test)

[1. 1. 1. ... 0. 0. 0.]


In [28]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def eval_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    data = [[accuracy, f1, precision, recall]]
    columns = ['accuracy', 'f1_score', 'precision', 'recall']
    eval_df = pd.DataFrame(data=data, columns=columns)
    return eval_df

result = eval_model(y_test, pred_test)

print(f'평가용 데이터에 대한 성능 평가 결과 : \n{result}')

평가용 데이터에 대한 성능 평가 결과 : 
   accuracy  f1_score  precision    recall
0  0.748524  0.753467   0.738941  0.768575
